In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
transaction_data=pd.read_csv('../dataset/transaction_data.csv')
demographic_data=pd.read_csv('../dataset/hh_demographic.csv')
product_data=pd.read_csv("../dataset/product.csv")

In [ ]:
def draw_missing_data_table(df):
    missing_count = df.isnull().sum().sort_values(ascending=False)
    missing_percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([missing_count, missing_percent], axis=1, keys=['Missing Count', 'Missing %'])
    return missing_data

In [ ]:
transaction_data.nunique()

In [ ]:
transaction_data=transaction_data[['household_key','BASKET_ID','DAY','PRODUCT_ID','QUANTITY','SALES_VALUE','WEEK_NO']]

In [ ]:
# days=pd.date_range(start='1/1/2017', end='12/12/2018').tolist()
# transaction_data.DAY=transaction_data.DAY.map(lambda x:days[x-1])

In [ ]:
age_map={'19-24':0,'25-34':1,'35-44':2,'45-54':3,'55-64':4,'65+':5}
demographic_data.AGE_DESC=demographic_data.AGE_DESC.map(age_map)

In [ ]:
income_map={'Under 15K':0,'15-24K':1,'25-34K':2,'35-49K':3,'50-74K':4,'75-99K':5,'100-124K':6,'125-149K':7,'150-174K':8,'175-199K':9,'200-249K':10,'250K+':11}
demographic_data.INCOME_DESC=demographic_data.INCOME_DESC.map(income_map)

In [ ]:
demographic_data.HOMEOWNER_DESC=demographic_data.HOMEOWNER_DESC.astype('string')

In [ ]:
marital_status_map={'A':'Married','B':'Single','U':'Unknown'}
demographic_data.MARITAL_STATUS_CODE=demographic_data.MARITAL_STATUS_CODE.map(marital_status_map)
demographic_data.MARITAL_STATUS_CODE=demographic_data.MARITAL_STATUS_CODE.astype('string')

In [ ]:
demographic_data['AGE_GROUP']=demographic_data.AGE_DESC
demographic_data['INCOME_GROUP']=demographic_data.INCOME_DESC

In [ ]:
demographic_data=demographic_data[['household_key','AGE_GROUP', 'MARITAL_STATUS_CODE', 'INCOME_GROUP', 'HOMEOWNER_DESC', 'HOUSEHOLD_SIZE_DESC','KID_CATEGORY_DESC']]

In [ ]:
demographic_data.dtypes

In [ ]:
values=transaction_data.household_key.sort_values()
household_keys=set(values.tolist())

# Many Household keys missing in the demografic data!!!
## Skipping of for now but need to definitly look at it later point of time.  

In [ ]:
product_data.dtypes

In [ ]:
product_data.BRAND=product_data.BRAND.astype('string')

In [ ]:
department_map={
    'GROCERY':'GROCERY',
    'DRUG GM':'DRUG GM',
    'PRODUCE':'PRODUCE',
    'COSMETICS':'COSMETICS',
    'NUTRITION':'NUTRITION',
    'MEAT':'MEAT',
    'MEAT-PCKGD':'MEAT-PCKGD',
    'DELI':'DELI',
    'PASTRY':'PASTRY',
    'FLORAL':'FLORAL',
    'SEAFOOD-PCKGD':'SEAFOOD-PCKGD',
    'MISC. TRANS.':'MISC. TRANS.',
    'SPIRITS':'SPIRITS',
    'SEAFOOD':'SEAFOOD',
    'GARDEN CENTER':'GARDEN CENTER',
    'RESTAURANT':'RESTAURANT',
    'TRAVEL & LEISUR':'TRAVEL & LEISUR',
    'RX':'OTHERS',
    'TOYS':'OTHERS',
    'POSTAL CENTER':'OTHERS',
    'PHOTO':'OTHERS',
    'KIOSK-GAS':'OTHERS',
    '':'OTHERS',
    'HBC':'OTHERS',
    'COUP/STR & MFG':'OTHERS',
    'FROZEN GROCERY':'GROCERY',
    'CHEF SHOPPE':'RESTAURANT',
    'CNTRL/STORE SUP':'OTHERS',
    'GM MERCH EXP':'OTHERS',
    'HOUSEWARES':'OTHERS',
    'AUTOMOTIVE':'OTHERS',
    'ELECT &PLUMBING':'OTHERS',
    'MISC SALES TRAN':'MISC. TRANS.',
    'CHARITABLE CONT':'MISC. TRANS.',
    'PROD-WHS SALES':'PRODUCE',
    'DELI/SNACK BAR':'DELI',
    'GRO BAKERY':'GROCERY',
    'DAIRY DELI':'DELI',
    'PORK':'MEAT',
    'MEAT-WHSE':'MEAT-PCKGD',
    'SALAD BAR':'PRODUCE',
    'VIDEO RENTAL':'ENTERTAINMENT',
    'VIDEO':'ENTERTAINMENT',
    'PHARMACY SUPPLY':'DRUG GM'
}

In [ ]:
product_data.DEPARTMENT=product_data.DEPARTMENT.map(department_map)
product_data.DEPARTMENT=product_data.DEPARTMENT.fillna("OTHERS")

In [ ]:
product_data=product_data.drop(['CURR_SIZE_OF_PRODUCT'],axis=1).rename(columns={'DEPARTMENT':'PRODUCT_TYPE','COMMODITY_DESC':'PRODUCT_DESC','SUB_COMMODITY_DESC':'SUB_PRODUCT_DESC'})

In [ ]:
total_data=pd.merge(transaction_data,product_data,on='PRODUCT_ID')
#total_data=transaction_data.set_index('PRODUCT_ID').join(product_data.set_index('PRODUCT_ID'))

In [ ]:
total_data[total_data.household_key==718]

In [ ]:
demand_products = total_data.PRODUCT_ID.value_counts().map(lambda x:x)
total_data=total_data[total_data.PRODUCT_ID.map(lambda x: demand_products[x])>500]

In [ ]:
train_data=total_data.copy()
train_data.drop(columns=['BASKET_ID', 'PRODUCT_DESC', 'SUB_PRODUCT_DESC', 'SALES_VALUE', 'BRAND', 'PRODUCT_TYPE', 'MANUFACTURER'],inplace=True)

In [ ]:
train_data.columns

In [ ]:
max_qty=pd.DataFrame(train_data.groupby(['household_key','PRODUCT_ID','DAY','WEEK_NO']).max()).reset_index().rename(columns={'QUANTITY':'MAX_QTY'})
min_qty=pd.DataFrame(train_data.groupby(['household_key','PRODUCT_ID','DAY','WEEK_NO']).min()).reset_index().rename(columns={'QUANTITY':'MIN_QTY'})
mean_qty=pd.DataFrame(train_data.groupby(['household_key','PRODUCT_ID','DAY','WEEK_NO']).mean()).reset_index().rename(columns={'QUANTITY':'MEAN_QTY'})

In [ ]:
train_data=train_data.merge(max_qty,on=['household_key','PRODUCT_ID','DAY','WEEK_NO'])
train_data=train_data.merge(min_qty,on=['household_key','PRODUCT_ID','DAY','WEEK_NO'])
train_data=train_data.merge(mean_qty,on=['household_key','PRODUCT_ID','DAY','WEEK_NO'])

In [ ]:
train_data['last_purchase_day']=train_data.groupby(['household_key','PRODUCT_ID'])['DAY'].shift()
train_data['days_since_last_purchase']=(train_data.DAY-train_data.last_purchase_day).fillna(-1).astype('int64')
train_data.drop('last_purchase_day',axis=1,inplace=True)

In [ ]:
# is_product=product_column.apply(str).astype('string').str.get_dummies()
# is_product.columns = ['buys_' + col for col in is_product.columns]

In [ ]:
# is_product=pd.get_dummies(train_data['PRODUCT_ID'],prefix = 'buys_',sparse=True)
# is_product.columns = ['buys_' + col for col in is_product.columns]

In [ ]:
# for id in set(train_data.PRODUCT_ID):
#     column_name='buys_'+str(id)
#     is_product=pd.DataFrame(train_data.PRODUCT_ID==id).rename(columns={'PRODUCT_ID':column_name})
#     train_data=pd.concat([train_data, is_product], axis=1)

In [ ]:
train_data=train_data.merge(product_data,on='PRODUCT_ID')

In [ ]:
train_data_demographic=pd.merge(demographic_data,train_data,on='household_key')
train_data.rename(columns={'household_key':'customer_id'},inplace=True)
train_data_demographic.rename(columns={'household_key':'customer_id'},inplace=True)

In [ ]:
train_data.to_csv("../data/total_data.csv")

In [ ]:
train_data_demographic.to_csv("../data/total_data_demographic.csv")

In [ ]:
train_data.PRODUCT_ID.value_counts()

In [ ]:
train_data['customer_id']